In [36]:
import pandas as pd
from pathlib import Path

TWITTER_BASE = Path('/data/blockchain-interoperability/blockchain-social-media/twitter-data/')

timestamps = pd.to_datetime(pd.read_pickle(TWITTER_BASE/'snapshots/timestamp_ms.pkl'),unit='ms')
user_id = pd.read_pickle(TWITTER_BASE/'snapshots/user_id.pkl')
cluster_id = pd.read_pickle(TWITTER_BASE/'snapshots/cluster_id.pkl')
df = pd.concat([user_id,timestamps,cluster_id],axis=1)
df = df[~df['timestamp_ms'].isna()]
df['user_id'] = df['user_id'].factorize()[0]

In [37]:
def get_user_appearances(tweets):
    cluster_counts = tweets['cluster_id'].value_counts()
    default_count = dict([(c,0) for c in range(6)])
    _, one_user_clusters = zip(*sorted({**default_count,**cluster_counts.to_dict()}.items()))
    return pd.Series(one_user_clusters,name='cluster_count')

def get_cluster_info(user_tweets): 
    return user_tweets.groupby('user_id').apply(get_user_appearances)

def user_cluster_over_time(interval = '1D'):
    save_path = TWITTER_BASE/f'cluster_movement/{interval}.npy'
    save_path.parent.mkdir(exist_ok = True, parents=True)
    if not save_path.is_file():
        change_over_time = df.resample(interval, on='timestamp_ms').apply(get_cluster_info)
        change_over_time.to_pickle(save_path)
    else:
        change_over_time = pd.read_pickle(save_path)
    return change_over_time

In [38]:
user_cluster_over_time('12H')
user_cluster_over_time('1D')

TypeError: Path.mkdir() got an unexpected keyword argument 'parent'

Main Idea: Visualize the overlaps in clusters as time passes